In [ ]:
!pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
!wget https://www.gutenberg.org/files/11/11-0.txt -O alice_in_wonderland.txt


--2025-02-10 05:28:26--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154573 (151K) [text/plain]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 150.95K  --.-KB/s    in 0.05s   

2025-02-10 05:28:27 (2.72 MB/s) - ‘alice_in_wonderland.txt’ saved [154573/154573]



In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load the GPT2 tokenizer and set pad_token
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token as the pad token

# Load the custom dataset
dataset = load_dataset("text", data_files={"train": "alice_in_wonderland.txt"})

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

print(tokenized_datasets)


Map:   0%|          | 0/3383 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 3383
    })
})


In [ ]:
model.save_pretrained("./gpt2-story-finetuned")
tokenizer.save_pretrained("./gpt2-story-finetuned")


('./gpt2-story-finetuned/tokenizer_config.json',
 './gpt2-story-finetuned/special_tokens_map.json',
 './gpt2-story-finetuned/vocab.json',
 './gpt2-story-finetuned/merges.txt',
 './gpt2-story-finetuned/added_tokens.json')

In [ ]:
# Modify training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-story-finetuned",
    evaluation_strategy="no",  # Disable evaluation
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    num_train_epochs=3,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="./gpt2-story-finetuned")

# Generate a creative story
result = generator("Once upon a time", max_length=200, num_return_sequences=1)
print(result[0]["generated_text"])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Once upon a time, what seems to you that the lightest person could make a living being with that power, you felt the power to control it."


A little later, with the arrival of the angel of death by his side, it was brought about he would be struck by lightning. He was, in the words of Professor Snape, once again in the grip of his curse with both men and women.


As a member of the Quidditch team, the one who brought him back - he was a half-elf, half-demon Lord himself and a fellow Hogwarts graduate - his curse was to destroy Professor Snape's wand and then to get Snape killed, to get him thrown out one night and then go and find out what had happened to Hermione Granger who was still at Hogwarts, just as she was. His curse was to put the magical boy as the new host, and, like the other members of the Quidditch team, the first to be killed was the
